This notebook will be used for the Capstone Project of the IBM Data Science Professional Certification Course.

In [2]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!
